<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/tensorflow/Simple_TensorFlow_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{simple-tensorflow} -->


# 🏋️‍♀️ W&B + 🌊 TensorFlow 2.x
Use Weights & Biases for machine learning experiment tracking, dataset versioning, and project collaboration.

<div><img /></div>

<img src="http://wandb.me/mini-diagram" width="650" alt="Weights & Biases" />

<div><img /></div>



## Get started, import libraries, use  GPU.

In [1]:
import tensorflow as tf
from tensorflow import keras

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Veysi ADIN 19 SEPT 2022
from pandas import read_csv
from numpy import set_printoptions
from sklearn import datasets, linear_model
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import random
# Using GPU
import os
import scipy.io as scpy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'  # Set to -1 if CPU should be used CPU = -1 , GPU = 0

gpus = tf.config.experimental.list_physical_devices('GPU')
cpus = tf.config.experimental.list_physical_devices('CPU')

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
elif cpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        logical_cpus= tf.config.experimental.list_logical_devices('CPU')
        print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPU")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


2022-10-05 10:41:58.160580: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-05 10:41:58.239144: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-05 10:41:58.261380: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-05 10:41:58.706722: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

1 Physical GPUs, 1 Logical GPUs


2022-10-05 10:41:59.789759: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-05 10:41:59.820791: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-05 10:41:59.820897: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-05 10:41:59.821497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

## Step 1️⃣: Import W&B and login

In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.login(key='0eeb933c285c4fbafee9f8d0f3d524022593e894')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: veysiadn. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/veysiadn/.netrc


True

# Prepare Dataset

In [3]:
data = scpy.loadmat("matlab15000.mat")
# Extracting x_train from the mat file dictionary.
x_data = data["XTrain"]
# Extracting y_train from the mat file dictionary.
y_data = data["unnamed"]
# Converting x_train and y_train to a numpy array.
x_data = np.array(x_data,dtype='float32')
y_train = np.array(y_data,dtype='float32')
# Verifying the shapes.
print(x_data.shape)
print(y_data.shape)

SEED = 99
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
# split into train test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=SEED)


x_val = x_test[:1500]
y_val = y_test[:1500]
x_test_to_use = x_test[1500:]
y_test_to_use = y_test[1500:]
print(f" {len(x_train), len(x_val), len(x_test_to_use)}")

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

(15000, 1000)
(15000, 1)
 (12000, 1500, 1500)


## Define the Model and the Training Loop

In [22]:
def make_conv_model():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Reshape((1000,1,1),name='Reshape')(inputs)
    x2 = tf.keras.layers.Conv2D(8,5,padding='same')(x1)
    x3= tf.keras.layers.Activation('relu')(x2)
    x4 = tf.keras.layers.MaxPooling2D(2,2, padding='same')(x3)
    x5 = tf.keras.layers.Flatten()(x4)
    outputs = tf.keras.layers.Dense(3,activation='softmax', name="output")(x5)
    return keras.Model(inputs=inputs, outputs=outputs)
def make_conv_model_paper():
    inputs=keras.Input(shape=(x_train.shape[1],))
    x0 = tf.keras.layers.Reshape((1000,1),name='Reshape')(inputs)
    x1=tf.keras.layers.Conv1D(32,7)(x0)
    x2=tf.keras.layers.BatchNormalization()(x1)
    x3=tf.keras.layers.Activation('relu')(x2)
    x4=tf.keras.layers.MaxPooling1D(pool_size=8)(x3)
    x5=tf.keras.layers.Conv1D(48,5)(x4)
    x6=tf.keras.layers.BatchNormalization()(x5)
    x7=tf.keras.layers.Activation('relu')(x6)
    x8=tf.keras.layers.MaxPooling1D(pool_size=4)(x7)
    x9=tf.keras.layers.MaxPooling1D(pool_size=4)(x8)
    x10=tf.keras.layers.Dropout(0.3)(x9)
    x11=tf.keras.layers.Flatten()(x10)
    x12=tf.keras.layers.Dense(32)(x11)
    x13=tf.keras.layers.Dense(32)(x12)
    outputs=tf.keras.layers.Dense(3, activation='softmax')(x13)
    return keras.Model(inputs=inputs, outputs=outputs)
def make_dense_model():
    inputs = keras.Input(shape=(x_train.shape[1],))
    x1 = tf.keras.layers.Flatten(input_shape=(1000,))(inputs)
    x2 = keras.layers.Dense(4, activation="relu")(x1)
    outputs = keras.layers.Dense(3, name="output")(x2)
    
    return keras.Model(inputs=inputs, outputs=outputs)

In [5]:
def train_step(x, y, model, optimizer, loss_fn, train_acc_metric):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)

    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y, logits)

    return loss_value

In [6]:
def test_step(x, y, model, loss_fn, val_acc_metric):
    val_logits = model(x, training=False)
    loss_value = loss_fn(y, val_logits)
    val_acc_metric.update_state(y, val_logits)

    return loss_value

## Step 2️⃣: Add `wandb.log` to your training loop

In [7]:
def train(train_dataset, val_dataset,  model, optimizer,
          train_acc_metric, val_acc_metric,
          epochs=10,  log_step=200, val_log_step=50):
  
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

        train_loss = []   
        val_loss = []

        # Iterate over the batches of the dataset
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            loss_value = train_step(x_batch_train, y_batch_train, 
                                    model, optimizer, 
                                    loss_fn, train_acc_metric)
            train_loss.append(float(loss_value))

        # Run a validation loop at the end of each epoch
        for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
            val_loss_value = test_step(x_batch_val, y_batch_val, 
                                       model, loss_fn, 
                                       val_acc_metric)
            val_loss.append(float(val_loss_value))
            
        # Display metrics at the end of each epoch
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))

        val_acc = val_acc_metric.result()
        print("Validation acc: %.4f" % (float(val_acc),))

        # Reset metrics at the end of each epoch
        train_acc_metric.reset_states()
        val_acc_metric.reset_states()

        # ⭐: log metrics using wandb.log
        wandb.log({'epochs': epoch,
                   'loss': np.mean(train_loss),
                   'acc': float(train_acc), 
                   'val_loss': np.mean(val_loss),
                   'val_acc':float(val_acc)})

# 👟 Run Training

## Step 3️⃣: Call `wandb.init` to start a run

This lets us know you're launching an experiment,
so we can give it a unique ID and a dashboard.

[Check out the official documentation here $\rightarrow$](https://docs.wandb.com/library/init)


In [25]:
# initialize wandb with your project name and optionally with configutations.
# play around with the config values and see the result on your wandb dashboard.
config = {
              "learning_rate": 0.001,
              "epochs": 100,
              "batch_size": 64,
              "log_step": 200,
              "val_log_step": 50,
           }

run = wandb.init(project='my-test-project', config=config, name='Dense_Model_Adam-1e-3')
config = wandb.config

# Initialize model.
model = make_dense_model()
model.summary()
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.Adam(learning_rate=config.learning_rate)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

train(train_dataset,
      test_dataset, 
      model,
      optimizer,
      train_acc_metric,
      val_acc_metric,
      epochs=config.epochs, 
      log_step=config.log_step, 
      val_log_step=config.val_log_step)

run.finish()  # In Jupyter/Colab, let us know you're finished!

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 1000)]            0         
                                                                 
 flatten_8 (Flatten)         (None, 1000)              0         
                                                                 
 dense_14 (Dense)            (None, 4)                 4004      
                                                                 
 output (Dense)              (None, 3)                 15        
                                                                 
Total params: 4,019
Trainable params: 4,019
Non-trainable params: 0
_________________________________________________________________

Start of epoch 0
Training acc over epoch: 0.6697
Validation acc: 0.8033

Start of epoch 1
Training acc over epoch: 0.8248
Validation acc: 0.8593

Start of epoch 2
Training acc over epoch: 

acc,▁▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████████████████████████
epochs,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
val_loss,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.96167
epochs,99
loss,0.11049
val_acc,0.93933
val_loss,0.22816


# 👀 Visualize Results

Click on the [**run page**](https://docs.wandb.ai/ref/app/pages/run-page)
link above to see your live results.

Or run the cell below to see an example dashboard rendered inside this notebook using [the `%wandb` magic](http://wandb.me/jupyter-interact-colab):

In [29]:
%wandb veysiadn/my-test-project/runs/<check-this-link-in-the-run-onverview> -h 720 

# 🧹 Sweep 101

Use Weights & Biases Sweeps to automate hyperparameter optimization and explore the space of possible models.

## [Check out Hyperparameter Optimization in TensorFlow using W&B Sweeps $\rightarrow$](http://wandb.me/tf-sweeps-colab)

### Benefits of using W&B Sweeps

* **Quick setup**: With just a few lines of code you can run W&B sweeps.
* **Transparent**: We cite all the algorithms we're using, and [our code is open source](https://github.com/wandb/client/tree/master/wandb/sweeps).
* **Powerful**: Our sweeps are completely customizable and configurable. You can launch a sweep across dozens of machines, and it's just as easy as starting a sweep on your laptop.

# 🎨 Example Gallery

See examples of projects tracked and visualized with W&B in our gallery of examples, [Fully Connected →](https://wandb.me/fc)

# 📏 Best Practices
1. **Projects**: Log multiple runs to a project to compare them. `wandb.init(project="project-name")`
2. **Groups**: For multiple processes or cross validation folds, log each process as a runs and group them together. `wandb.init(group='experiment-1')`
3. **Tags**: Add tags to track your current baseline or production model.
4. **Notes**: Type notes in the table to track the changes between runs.
5. **Reports**: Take quick notes on progress to share with colleagues and make dashboards and snapshots of your ML projects.

## 🤓 Advanced Setup
1. [Environment variables](https://docs.wandb.com/library/environment-variables): Set API keys in environment variables so you can run training on a managed cluster.
2. [Offline mode](https://docs.wandb.com/library/technical-faq#can-i-run-wandb-offline): Use `dryrun` mode to train offline and sync results later.
3. [On-prem](https://docs.wandb.com/self-hosted): Install W&B in a private cloud or air-gapped servers in your own infrastructure. We have local installations for everyone from academics to enterprise teams.
4. [Artifacts](http://wandb.me/artifacts-colab): Track and version models and datasets in a streamlined way that automatically picks up your pipeline steps as you train models.